# Fuzzy matching Attractions:
This is the third sub-module of the `activities_recommendation` module. Firstly we define the purpose of the module and the challenges, then we will dive into the features of the sub-module and then will walkthrough the code.

## Context:
As mentioned earlier, the aim for the 4-level sub-module is to map Viator activities to the TripAdvisor attraction data. In the `pre-processing` sub-module, we splitted the Viator activities in mapped and unmapped activities (ref. TripAdvisor activities). In the `NER_Extraction` module, we extracted the potential attraction names out of the activity description. Now we will do a fuzzy matching with the TripAdvisor attraction data, finally mapping the activities with the attraction. There is a later stage also, where we do an inner join on this data to finally get the desired `.csv` output

## Initialization:
We import the required packages and convert the string based lists (due to reading from `.csv`) to python lists, we also add a new column called `possible_attractions`, which for now has an empty list, but in the end will have the desired mappings.

In [28]:
VIATOR_ACT_NER_DONE = 'viator_act_ner_done.csv'
TA_ATT = '../ta_attractions.json'
VIATOR_ACT_WITH_POSSIBLE_ATT = 'viator_act_with_possible_att.csv'

import pandas as pd
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None
act_df = pd.read_csv(VIATOR_ACT_NER_DONE)

In [29]:
# convert the string based lists to python lists
import ast
act_df['locations'] = act_df['locations'].apply(lambda llist: ast.literal_eval(llist))
act_df['organizations'] = act_df['organizations'].apply(lambda llist: ast.literal_eval(llist))

In [30]:
# add an empty column with lists to the activity data frame
import numpy as np
act_df['possible_attractions'] = np.empty((len(act_df), 0)).tolist()
act_df.head(2)

,Rank,ProductType,ProductCode,ProductName,Introduction,ProductText,Special,Duration,Commences,ProductImage,ProductImageThumb,DestinationID,Continent,Country,Region,City,IATACode,Group1,Category1,Subcategory1,Group2,Category2,Subcategory2,Group3,Category3,Subcategory3,ProductURL,PriceAUD,PriceNZD,PriceEUR,PriceGBP,PriceUSD,PriceCAD,PriceCHF,PriceNOK,PriceJPY,PriceSEK,PriceHKD,PriceSGD,PriceZAR,AvgRating,AvgRatingStarURL,BookingType,VoucherOption,locations,organizations,possible_attractions
0,4,SITours_NEW,3951WESTDLX,Grand Canyon West Rim and Hoover Dam Tour from...,Hit the highway out of Las Vegas and spend the...,Hit the highway out of Las Vegas and spend the...,0,12 hours,"Las Vegas, United States",https://media.tacdn.com/media/attractions-spli...,https://media.tacdn.com/media/attractions-spli...,684.0,Northern America,United States,Nevada,Las Vegas,LAS,"Air, Helicopter & Balloon Tours",Helicopter Tours,Helicopter Tour,Tours & Sightseeing,Bus & Minivan Tours,Bus Tour,Outdoor Activities,"4WD, ATV & Off-Road Tours",Adventure Tour,http://www.partner.viator.com/en/66575/tours/L...,183.76,198.86,"110,02",98.28,118.19,168.25,117.43,"1 209,71","13,007",1172.09,938.95,171.65,2214.89,4.5,http://www.partner.viator.com/images/stars/red...,Freesale,VOUCHER_E,"[grand canyon, grand canyon skywalk]",[],[]
1,5,SITours_NEW,2280LI_5H,Grand Canyon 4-in-1 Helicopter Tour,Take the ultimate Grand Canyon tour! You'll fl...,Take the ultimate Grand Canyon tour! You'll fl...,0,6 hours 30 minutes,"Las Vegas, United States",https://media.tacdn.com/media/attractions-spli...,https://media.tacdn.com/media/attractions-spli...,684.0,Northern America,United States,Nevada,Las Vegas,LAS,"Air, Helicopter & Balloon Tours",Helicopter Tours,Helicopter Tour,Tours & Sightseeing,Self-guided Tours & Rentals,Self Guided Tours,Tours & Sightseeing,Full-day Tours,Day Tour,http://www.partner.viator.com/en/66575/tours/L...,963.96,1043.14,"577,16",515.54,619.99,882.61,615.99,"6 345,79","68,230",6148.42,4925.44,900.43,11618.64,5.0,http://www.partner.viator.com/images/stars/red...,FreesaleOnRequest,VOUCHER_E,"[grand canyon, colorado river, west rim, grand...",[],[]


In [ ]:
hello world 
hello world from the earth

## The Fuzzy Matching Algorithm
***Input***: An array of `locations` or `organizations` (extracted via `NER_Extraction` module) and an `attraction_name` (actually a complete record for `attraction-data`)

***Output***: An array of tuples. (`ner_name`, `matched_attraction_name`, `percentage_of_match`)

***Algorithm:***
For every `ner_name` and `att_name` we check the number of words in both of them and then generate the `n_grams` for the one which has greater number of words, `n` (`n` value in `n_grams`) being the no. of words of the other variable. Then for each `n_gram` we check the `fuzz.ratio` (which is simply the edit distance) of it with the other variable (both have the same no. of words), and finally check the `max_fuzz_ratio`. This would be more clear with an example:

#### Example:
***Input***: `['grand canyon']` (location) and `"Grand Canyon National Park"`
- since number of words in `'grand canyon' < "Grand Canyon Tour by EX Travels!"`, so `n = 2` and `n_grams` for `att_name` would be generated.

- `n_grams = [('Grand', 'Canyon'), ('Canyon', 'National'), ('National', 'Park')]`

- Now we join words for each tuple and do a `fuzz.match`. In this case, it would be `> 95%` match and thus, this is a possible attraction for the activity.

#### Use Cases:
**What about something like `Gambhir River` and `Ganga River at Varanasi`?**
- If we hadn't generated the `n_grams`, doing a `fuzz.partial_ratio` would give almost `100%` match. Thus, producing a false positive. 
- But using `n_grams` eliminates this as now total comparisions would be made, and we won't lose accuracy since only bigrams would be checked.

**And what about `Ganga River` and `Dashashwamedh Ghat, Ganga River`?**
- If we use a `fuzz.partial_ratio` here, this will give an accurate mapping, but that would produce a false positive for the use case mentioned above. However, if we use a simple `fuzz.ratio` here, we won't be able to produce accurate result and will miss this mapping.

Thus, we generate the `n_grams` and then use a simple `fuzz.ratio` over this. Then we compare the `max_fuzz_ratio` with the `THRESHOLD_RATIO`.

#### Time Complexity:
Since the `att_name` is on average 5 words long and `location` about 2 words, this algo works in `O(1)` time. Though, this isn't the computation bottleneck, but I used `rapidfuzz` which is written in `C++`, thus works really fast. As the number of records grow, this tackles the `O(1)` time bottleneck.

In [31]:
# from fuzzywuzzy import fuzz
from rapidfuzz import fuzz
from nltk import ngrams
THRESHOLD_RATIO = 70

def find_max_fuzz_ratio(str1, str2):
    """Takes two strings and outputs max. fuzz ratio between them by comparing the n_grams of one with the other"""
    
    if len(str1.split()) < len(str2.split()):
        n = len(str1.split())
        n_grams = ngrams(str2.split(), n)
        name_to_check = str1
    else:
        n = len(str2.split())
        n_grams = ngrams(str1.split(), n)
        name_to_check = str2
        
    max_ratio = -1
    for gram in n_grams:
        
        # create the word by joining the tuple elements
        att_name = ' '.join(gram)
        ratio = fuzz.ratio(att_name, name_to_check)
        max_ratio = max(max_ratio, ratio)
    return max_ratio

def check_mappings(att_name, act_row):
    # Since same row would be accessed by multiple processes of a pool, we need a lock to pass without race conditions
    # however, I used the lock to add debugging statements inside the code.
    with lock:
        for ner_loc in act_row['locations']:
            ratio = find_max_fuzz_ratio(att_name, ner_loc)
            
            if ratio > THRESHOLD_RATIO:
                return ((att_name, ner_loc, ratio), True)

        for ner_org in act_row['organizations']:
            
            ratio = find_max_fuzz_ratio(ner_org, att_name)
            if ratio > THRESHOLD_RATIO:
                return ((att_name, ner_org, ratio), True)
            
        return (att_name, False)

## Multi-processing Wrapper
- To speed up the matching with the computation power we have, we have this wrapper (coded like a caller, but providing essential functionality to the main fuzzy `check_mappings` algorithm). For this, we define an initializer which will pass a global lock to the function to be used by the child processes. The wrapper `parallelize_mapping` splits up the dataframe passed to it into the number of cores and then let's the worker do their job for each split parallely.

- Moreover, an essential filter is also provided at this wrapper, which extracts the relevant attractions to check for mapping based on the `possible_city` (which is the second last element of the `parents` array), and the `City` column. So essentially, we check the attractions of a city for a given location (which is of that city). This brings down the number of comparisions from about **100,000** to a **Few Hundreds** (lightening fast!, when it has to be repeated over half a million times).

In [32]:
# Multiprocessing wrapper
from multiprocessing import Pool, Lock

# Initializer, with a global lock for all the child processes
def init(l):
    global lock
    lock = l

def parallelize_mapping(row, function, test_df, n_cores=4):
    
    # filter based on the possible_city and the City key from the viator data
    test_df = test_df.loc[test_df['possible_city'] == (row['City']).lower()]
    
    # split the dataframe
    test_df_split = np.array_split(test_df, n_cores)
    
    # lock to be passed to the initializer
    l = Lock()
    
    # add lock as an argument to each of the tuple
    for idx in range(len(test_df_split)):
        test_df_split[idx] = (row, test_df_split[idx])
    
    # initialize pool of workers with lock l
    pool = Pool(n_cores, initializer=init, initargs=(l,))
    
    # starmap the pool to the function and get the results, results would be a list of lists
    results = pool.starmap(function, test_df_split)
    pool.close()
    pool.join()
    
    # unpack the results to fit them in a single list -- this is the result for an activity
    return [item for sublist in results for item in sublist]


In [33]:
# We use boolean indexing to get rid of attraction names which are not mapped
def sanitize(att_tuple):
    return att_tuple[0]

def map_attractions(row, att_df):
    
    # since this is applied to attractions data we will have an output corresponding to each att_name
    att_mapped = att_df['name'].apply(check_mappings, act_row=row)
    
    # We want to keep only those att_names which are mapped, i.e. for the tuple[1] == True
    att_mapped = att_mapped[att_mapped.apply(lambda x: x[1] == True)]
    
    # Sanitize by removing the boolean index
    att_mapped = att_mapped.apply(sanitize)
    
    # drop duplicates for the same location, as this is redundant data
    att_mapped.drop_duplicates(inplace=True)
    
    # return, as list
    return att_mapped.to_list()

## Driver with masks:
Now is the time to drive the code, but we have to apply some masks over the data to filter unnecessary data. Since the abstract of attraction data is to provide the names of attraction, we need to filter entries which are actually operator names (since that is in the Viator data, and is an activity essentially). So, The Driver does the following things:
- Masks the attraction data to filter the records with keywords in the `to_exclude` list, by regex matching and vectorization (**This filters almost 7% attractions**)
- Create a column `possible_city` which would be later used to filter the dataframe for a certain activity based on its `City`, this is the second last element of the `parents` list.
- Create an intermediate column called `to_append` in which results for each of the iteration of the reader would be stored. Later on, this would be concatened with `possible_attractions`.
- In the end, we will have `possible_attractions` which is a list of tuples `(ner_name, attraction_name, percentage_matching)`.

In [34]:
import time

reader = pd.read_json(TA_ATT, lines=True, chunksize=50000)
to_exclude = ['tours', 'tour', 'adventure', 'adventures', 'closed']

start_time = time.time()
for att_df in reader:
    
    # Regex mask to filter attractions with the given keywords
    mask = att_df['name'].str.lower().str.contains(r'\b(?:{})\b'.format('|'.join(to_exclude)))
    
    # apply the mask
    att_df = att_df[~mask]
    print('Processing sanitized json chunk... Shape: ', att_df.shape)
    
    # add a possible city
    att_df['possible_city'] = att_df['parents'].apply(lambda parents: parents[-2].lower())
    
    # create an intermediate column
    act_df['to_append'] = act_df.apply(parallelize_mapping, function=map_attractions, test_df=att_df, axis=1)
    
    # finally, gather the data in the possible_attraction column
    act_df['possible_attractions'] = act_df['possible_attractions'] + act_df['to_append']  
    
end_time = time.time()

print('Time taken(s): ', end_time - start_time)
act_df.head(3)

Processing sanitized json chunk... Shape:  (45585, 21)
Processing sanitized json chunk... Shape:  (46770, 21)
Processing sanitized json chunk... Shape:  (47760, 21)
Processing sanitized json chunk... Shape:  (48132, 21)
Processing sanitized json chunk... Shape:  (47435, 21)
Processing sanitized json chunk... Shape:  (47223, 21)
Processing sanitized json chunk... Shape:  (45243, 21)
Processing sanitized json chunk... Shape:  (48479, 21)
Processing sanitized json chunk... Shape:  (46889, 21)
Processing sanitized json chunk... Shape:  (44166, 21)
Processing sanitized json chunk... Shape:  (46829, 21)
Processing sanitized json chunk... Shape:  (46549, 21)
Processing sanitized json chunk... Shape:  (31857, 21)
Time taken(s):  955.0235593318939


,Rank,ProductType,ProductCode,ProductName,Introduction,ProductText,Special,Duration,Commences,ProductImage,ProductImageThumb,DestinationID,Continent,Country,Region,City,IATACode,Group1,Category1,Subcategory1,Group2,Category2,Subcategory2,Group3,Category3,Subcategory3,ProductURL,PriceAUD,PriceNZD,PriceEUR,PriceGBP,PriceUSD,PriceCAD,PriceCHF,PriceNOK,PriceJPY,PriceSEK,PriceHKD,PriceSGD,PriceZAR,AvgRating,AvgRatingStarURL,BookingType,VoucherOption,locations,organizations,possible_attractions,to_append
0,4,SITours_NEW,3951WESTDLX,Grand Canyon West Rim and Hoover Dam Tour from...,Hit the highway out of Las Vegas and spend the...,Hit the highway out of Las Vegas and spend the...,0,12 hours,"Las Vegas, United States",https://media.tacdn.com/media/attractions-spli...,https://media.tacdn.com/media/attractions-spli...,684.0,Northern America,United States,Nevada,Las Vegas,LAS,"Air, Helicopter & Balloon Tours",Helicopter Tours,Helicopter Tour,Tours & Sightseeing,Bus & Minivan Tours,Bus Tour,Outdoor Activities,"4WD, ATV & Off-Road Tours",Adventure Tour,http://www.partner.viator.com/en/66575/tours/L...,183.76,198.86,"110,02",98.28,118.19,168.25,117.43,"1 209,71","13,007",1172.09,938.95,171.65,2214.89,4.5,http://www.partner.viator.com/images/stars/red...,Freesale,VOUCHER_E,"[grand canyon, grand canyon skywalk]",[],"[(Grand Canyon & Beyond, grand canyon, 83.3333...",[]
1,5,SITours_NEW,2280LI_5H,Grand Canyon 4-in-1 Helicopter Tour,Take the ultimate Grand Canyon tour! You'll fl...,Take the ultimate Grand Canyon tour! You'll fl...,0,6 hours 30 minutes,"Las Vegas, United States",https://media.tacdn.com/media/attractions-spli...,https://media.tacdn.com/media/attractions-spli...,684.0,Northern America,United States,Nevada,Las Vegas,LAS,"Air, Helicopter & Balloon Tours",Helicopter Tours,Helicopter Tour,Tours & Sightseeing,Self-guided Tours & Rentals,Self Guided Tours,Tours & Sightseeing,Full-day Tours,Day Tour,http://www.partner.viator.com/en/66575/tours/L...,963.96,1043.14,"577,16",515.54,619.99,882.61,615.99,"6 345,79","68,230",6148.42,4925.44,900.43,11618.64,5.0,http://www.partner.viator.com/images/stars/red...,FreesaleOnRequest,VOUCHER_E,"[grand canyon, colorado river, west rim, grand...",[],"[(Grand Canyon & Beyond, grand canyon, 83.3333...",[]
2,10,SITours_NEW,5022MOUDIN,Moulin Rouge Paris Dinner and Show Ticket,Enjoy an evening dinner show at the Moulin Rou...,Enjoy an evening dinner show at the Moulin Rou...,0,4 hours,"Paris, France",https://media.tacdn.com/media/attractions-spli...,https://media.tacdn.com/media/attractions-spli...,479.0,Western Europe,France,Ile-de-France,Paris,CDG,"Shows, Concerts & Sports",Cabaret,Cabaret,"Shows, Concerts & Sports","Theater, Shows & Musicals",Show,"Shows, Concerts & Sports",Dinner Packages,Dinner and Show,http://www.partner.viator.com/en/66575/tours/P...,322.92,349.41,"190,00",172.70,211.53,295.64,206.31,"2 125,76","22,856",2059.70,1609.57,301.61,3891.78,4.5,http://www.partner.viator.com/images/stars/red...,OnRequest,VOUCHER_E,[moulin rouge],[moulin rouge],"[(La Machine du Moulin Rouge, moulin rouge, 83...",[]


In [35]:
# export to csv file
act_df.to_csv(VIATOR_ACT_WITH_POSSIBLE_ATT, index=False, encoding='utf-8')

df.explode('possible_att').explode('locations').drop